In [1]:
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
from datetime import date
from openpyxl import load_workbook


url = 'https://www.worldometers.info/coronavirus/'
req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
webpage = urlopen(req).read()

soup = BeautifulSoup(webpage, "html.parser")


table = soup.find('table')
table_rows = table.find_all('tr')

res = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    res.append(row)   

In [2]:
#res

In [3]:
x = """idx
Country
Total Cases
New Cases
Total Deaths
New Deaths
Total Recovered
New Recovered
Active Cases
Critical
Cases/1M pop
Deaths/1M pop
Total Tests
Tests/1M pop
Population
Continent
1 Case every X ppl
1 Death every X ppl
1 Test every X ppl""".split('\n')


data = pd.DataFrame(res,columns = x)

In [4]:
#print(data)

In [5]:
data.drop(0,inplace=True)

data.set_index('Country',inplace=True)

# Removing 'world' row
data.drop(['World','Total:'],inplace=True)

continents = [i for i in data.index if '\n' in i]
data.drop(continents,inplace=True)

In [6]:
data.drop("""idx
Population
Continent
1 Case every X ppl
1 Death every X ppl
1 Test every X ppl
New Recovered""".split('\n'),axis=1,inplace=True)

In [7]:
#pathCoronaOld = r'C:\Users\97250\Desktop\coronaWeb\coronaWorld.xlsx'
#oldCountries = pd.read_excel(pathCoronaOld, sheet_name=0)['Country'].values
#newCountries = data.index

In [8]:
def commaRemover(x):
    
    if type(x) == str:
        
        if x == ' ' or x == '' or x == 'N/A' : # in list is better
            x = 0
                    
        else:
            x = x.replace(',','').replace("+","")
            x = float(x)
    
    return x
    
for i in data.columns:
    data[i] = data[i].apply(commaRemover)

#adding new columns
#have to watch infinties of no cases are reported

#perhaps this:
data['Deaths/Cases'] =  data[['Total Deaths','Total Cases']].apply(lambda x : round(int(x[0])/int(x[1]),2) if x[1] !='' else 0 ,axis=1)
data['Tests/Cases'] =  data[['Total Tests','Total Cases']].apply  (lambda x : round(x[0]/x[1],2) if x[1] !=0 else 1000 ,axis=1)

# Fill NA's
data.fillna(value = 0,inplace =True)

In [9]:
data

,Total Cases,New Cases,Total Deaths,New Deaths,Total Recovered,Active Cases,Critical,Cases/1M pop,Deaths/1M pop,Total Tests,Tests/1M pop,Deaths/Cases,Tests/Cases
Country,,,,,,,,,,,,,
USA,9802374.0,1019.0,239842.0,13.0,6293132.0,3269400.0,18045.0,29554.0,723.0,152052680.0,458439.0,0.02,15.51
India,8364086.0,674.0,124354.0,0.0,7711809.0,527923.0,8944.0,6040.0,90.0,114208384.0,82480.0,0.01,13.65
Brazil,5590941.0,0.0,161170.0,0.0,5064344.0,365427.0,8318.0,26238.0,756.0,21900000.0,102777.0,0.03,3.92
Russia,1712858.0,19404.0,29509.0,292.0,1279169.0,404180.0,2300.0,11735.0,202.0,63000000.0,431636.0,0.02,36.78
France,1543321.0,0.0,38674.0,0.0,122662.0,1381985.0,4089.0,23626.0,592.0,16819377.0,257477.0,0.03,10.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...
MS Zaandam,9.0,0.0,2.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.22,0.00
Anguilla,3.0,0.0,0.0,0.0,3.0,0.0,0.0,199.0,0.0,1820.0,120938.0,0.00,606.67
Marshall Islands,2.0,0.0,0.0,0.0,0.0,2.0,0.0,34.0,0.0,0.0,0.0,0.00,0.00


In [10]:

today = str(date.today())

path = r'C:\Users\97250\Desktop\coronaWeb\coronaWorld.xlsx'
book = load_workbook(path)

### checks if no sheet from today
last_sheet = book.get_sheet_names()[-1]
if last_sheet==today:
    book.remove_sheet(book.get_sheet_by_name(last_sheet))

writer = pd.ExcelWriter(path, engine = 'openpyxl')
writer.book = book    
data.to_excel(writer ,sheet_name = today) #the sheet name will be the str(current date)
writer.save()
writer.close()

<ipython-input-10-5962503f643f>:7: DeprecationWarning: Call to deprecated function get_sheet_names (Use wb.sheetnames).
  last_sheet = book.get_sheet_names()[-1]


In [11]:
#checks sheets
file = pd.read_excel(path, sheet_name=None)

sheets = [i for i in file][-10:]

sheets

['2020-05-08',
 '2020-05-09',
 '2020-05-10',
 '2020-05-12',
 '2020-05-14',
 '2020-05-15',
 '2020-05-18',
 '2020-10-19',
 '2020-10-25',
 '2020-11-05']